In [8]:
#Forest Layer
from sklearn.base import ClassifierMixin
from sklearn.ensemble import RandomForestClassifier,ExtraTreesClassifier,VotingClassifier
from sklearn.model_selection import KFold
import numpy as np
import copy
class deepForestLayer(ClassifierMixin):
    """A Really hacky and WIP implementation of the layers required for a Cascade Forest"""
    def __init__(self, n_nodes, nClasses=1,output=False):
        self.output=output
        self.nClasses = nClasses
        self.n_nodes= n_nodes
        nrfs = int(n_nodes/2)
        nefs = n_nodes - nrfs
        self.estimators = []
        self.final_voters = []
        for i in range(nefs):
            self.estimators.append(('ET'+str(i),ExtraTreesClassifier(n_estimators=1000,min_samples_leaf=10, n_jobs=-1)))
            
        for i in range(nrfs):
            self.estimators.append(('RF'+str(i),RandomForestClassifier(n_estimators=1000,min_samples_leaf=10,n_jobs=-1)))
        self.voter = VotingClassifier(estimators=self.estimators, voting='soft')

    def fit_Kfold(self,X_train, X_test,y_train, y_test):
        """This function implements the growing and validation to determine number of layers required"""
        
        fold = KFold() # 3-Fold CV
        train_preds = np.empty((3,X_train.shape[0],self.nClasses*self.n_nodes))
        train_preds[:] = np.nan
        est_preds = np.empty((3,X_test.shape[0],self.nClasses*self.n_nodes))
        est_preds[:] = np.nan
        i=0
        for train_idx, test_idx in fold.split(X_train):
            self.voter.fit(X_train[train_idx],y_train[train_idx]) #Fit each of the estimators to our data
            
            #voter.transform has shape number of estimators (4 in this case) x no of samples x number of classes.
            #Insample is the transform output reshaped to have shape no of samples x number of classes*number of estimators
            insample = self.voter.transform(X_train[train_idx]).swapaxes(0,1).reshape((X_train[train_idx].shape[0],-1))
            outsample = self.voter.transform(X_test).swapaxes(0,1).reshape((X_test.shape[0],-1))
            #Insample is the training error, outsample is the validation error.
            
            train_preds[i,train_idx] = insample.copy()
            est_preds[i] = outsample.copy()
            
            i+=1
        #As I used KFold, train_preds and est_preds have two valid entries and one nan entry per data point
        #average this dimension so we get one probability prediction per data point
        return np.nanmean(train_preds,axis=0),np.nanmean(est_preds,axis=0)
        
    def fit(self, X,y):
        """This function does a full fit once the number of layers has been decided"""
        fold = KFold()
        #Create 3 models, each fitted on a fold of the data.
        #This is only way I can think of getting required output at prediction stage.
        for train_idx, test_idx in fold.split(X):
            clf=VotingClassifier(estimators=copy.deepcopy(self.estimators), voting='soft')
            clf.fit(X[train_idx], y[train_idx])
            self.final_voters.append(clf)
        
    def predict(self, X):
        """Make predictions, using models fitted by KFold"""
        preds = np.zeros((len(self.final_voters),X.shape[0],self.nClasses))
        for i in range(len(self.final_voters)):
            preds[i]=self.final_voters[i].predict_proba(X)
        return np.mean(preds,axis=0)
        
    

In [17]:
#DeepForest
import numpy as np
from sklearn.base import ClassifierMixin, BaseEstimator
from sklearn.model_selection import train_test_split
# from forestlayer import deepForestLayer
from sklearn.metrics import accuracy_score

class DeepForest(ClassifierMixin,BaseEstimator):
    def __init__(self, width=4):
        self.width= width
        self.layers=[]
        self.layerScores=[]

    def fit(self, X,y):
        
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)
        n=len(np.unique(y))
        last_score = -np.inf
        previous_est =  np.empty((X_test.shape[0],0))
        previous_pred = np.empty((X_train.shape[0],0))
        i=1
        while True:
            print ("layer "+str(i))
            layer = deepForestLayer(4,nClasses=n)
            pred, est = layer.fit_Kfold(np.concatenate([X_train,previous_pred],axis=1), np.concatenate([X_test,previous_est],axis=1), y_train,y_test)
            yest = np.argmax(np.mean(est.reshape((X_test.shape[0],-1, n)).swapaxes(0,1),axis=0),axis=1)
            score = accuracy_score(y_test, yest)
            print (score)
            if score<=last_score:
                break
            last_score=score
            previous_est=est
            previous_pred=pred
            i+=1
        print ("Selected "+str(i-1)+" layers")
        self.full_fit(X,y,i-1)
        
    def full_fit(self, X,y, layers):
         n=len(np.unique(y))
         previous_pred = np.empty((X.shape[0],0))
         for l in range(layers):
             layer = deepForestLayer(4, nClasses=n)
             newX = np.concatenate([X,previous_pred],axis=1)
             layer.fit(newX,y)
             previous_pred = layer.predict(newX)
             self.layers.append(layer)
         print ("fitted!!")
             
    def predict(self, X):
        previous_pred = np.empty((X.shape[0],0))
        for layer in self.layers:
            newX = np.concatenate([X,previous_pred],axis=1)
            previous_pred = layer.predict(newX)
        return np.argmax(previous_pred,1)
            
            
    

In [18]:
#!/usr/bin/env python2
# -*- coding: utf-8 -*-
"""
Created on Wed Mar 15 22:37:57 2017

@author: chris
"""

from sklearn.model_selection import GridSearchCV
from sklearn.datasets import load_digits
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
# from deepforest import DeepForest

X,y = load_digits(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

df = DeepForest()
df.fit(X_train, y_train)

base = RandomForestClassifier()
gs = GridSearchCV(base, {'n_estimators':[2000]})
gs.fit(X_train, y_train)


mypred= df.predict(X_test)
print ("Deep Forest:",accuracy_score(y_test, mypred))
print ("RF: ",accuracy_score(y_test, gs.predict(X_test)))



layer 1


C:\Users\Petrofac\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:426: FutureWarning: You should specify a value for 'n_splits' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(NSPLIT_WARNING, FutureWarning)
C:\Users\Petrofac\Anaconda3\lib\site-packages\sklearn\ensemble\voting_classifier.py:307: DeprecationWarning: 'flatten_transform' default value will be changed to True in 0.21. To silence this warning you may explicitly set flatten_transform=False.
  DeprecationWarning)
C:\Users\Petrofac\Anaconda3\lib\site-packages\sklearn\ensemble\voting_classifier.py:307: DeprecationWarning: 'flatten_transform' default value will be changed to True in 0.21. To silence this warning you may explicitly set flatten_transform=False.
  DeprecationWarning)
C:\Users\Petrofac\Anaconda3\lib\site-packages\sklearn\ensemble\voting_classifier.py:307: DeprecationWarning: 'flatten_transform' default value will be changed to True in 0

0.9409722222222222
layer 2


C:\Users\Petrofac\Anaconda3\lib\site-packages\sklearn\ensemble\voting_classifier.py:307: DeprecationWarning: 'flatten_transform' default value will be changed to True in 0.21. To silence this warning you may explicitly set flatten_transform=False.
  DeprecationWarning)
C:\Users\Petrofac\Anaconda3\lib\site-packages\sklearn\ensemble\voting_classifier.py:307: DeprecationWarning: 'flatten_transform' default value will be changed to True in 0.21. To silence this warning you may explicitly set flatten_transform=False.
  DeprecationWarning)
C:\Users\Petrofac\Anaconda3\lib\site-packages\sklearn\ensemble\voting_classifier.py:307: DeprecationWarning: 'flatten_transform' default value will be changed to True in 0.21. To silence this warning you may explicitly set flatten_transform=False.
  DeprecationWarning)
C:\Users\Petrofac\Anaconda3\lib\site-packages\sklearn\ensemble\voting_classifier.py:307: DeprecationWarning: 'flatten_transform' default value will be changed to True in 0.21. To silence thi

0.9548611111111112
layer 3


...

0.9548611111111112
Selected 2 layers


C:\Users\Petrofac\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:426: FutureWarning: You should specify a value for 'n_splits' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(NSPLIT_WARNING, FutureWarning)


fitted!!


C:\Users\Petrofac\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


Deep Forest: 0.95
RF:  0.9777777777777777
